In [1]:
pip install transformers datasets torch scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

df = pd.read_csv("/kaggle/input/dataset-2/balanced_transformed_intent_dataset.csv")

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

In [3]:
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
intents = df["intent"].unique()
label_encoder = LabelEncoder()
label_encoder.fit(intents)

intent_to_texts = {}
for intent in intents:
    intent_to_texts[intent] = df[df['intent'] == intent]['input_text'].tolist()

df['intent_encoded'] = label_encoder.transform(df['intent'])

encoder_data = {
    'label_encoder': label_encoder,
    'intent_to_texts': intent_to_texts,
    'encoded_queries': df[['input_text', 'intent_encoded']].to_dict('records')
}

with open("text_intent_encoder.pkl", "wb") as f:
    pickle.dump(encoder_data, f)
with open("text_intent_encoder.pkl", "rb") as f:
    loaded_data = pickle.load(f)
    
print("Saved encoder can map these intents:", loaded_data['label_encoder'].classes_)
print("Example mapping for first intent:", 
      loaded_data['intent_to_texts'][loaded_data['label_encoder'].classes_[0]][:1])

Saved encoder can map these intents: ['bug_identification' 'code_conversion' 'code_generation' 'code_summary'
 'design_explanation' 'function_explanation' 'generate_docstring' 'misc'
 'refactor_suggestion' 'schema_inference' 'visualization']
Example mapping for first intent: ['Can you generate a brief explanation for the following Python 3 code def _make_eof_intr(): global _EOF, _INTR if (_EOF is not None) and (_INTR is not None): return # inherit EOF and INTR definitions from controlling process. try: from termios import VEOF, VINTR fd = None for name in \'stdin\', \'stdout\': stream = getattr(sys, \'__%s__\' % name, None) if stream is None or not hasattr(stream, \'fileno\'): continue try: fd = stream.fileno() except ValueError: continue if fd is None: # no fd, raise ValueError to fallback on CEOF, CINTR raise ValueError("No stream has a fileno") intr = ord(termios.tcgetattr(fd)[6][VINTR]) eof = ord(termios.tcgetattr(fd)[6][VEOF]) except (ImportError, OSError, IOError, ValueError, ter

In [4]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(
        examples["input_text"],
        padding="max_length",
        truncation=True,   
        max_length=512     
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
def encode_labels(examples):
    return {"labels": label_encoder.transform(examples["intent"])}

tokenized_dataset = tokenized_dataset.map(encode_labels, batched=True)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [7]:
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer

model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=len(label_encoder.classes_)  
)

2025-07-04 07:38:57.699306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751614738.096072      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751614738.203557      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import (
    RobertaForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    EarlyStoppingCallback
)
import numpy as np
from sklearn.metrics import accuracy_score
import torch
total_steps = len(tokenized_dataset["train"]) 
warmup_steps = int(0.1 * total_steps)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

training_args = TrainingArguments(
    output_dir="./intent_classifier",
    eval_strategy="epoch", 
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    num_train_epochs=5,
    save_total_limit=3,
    logging_dir="./logs",
    logging_steps=50,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=warmup_steps,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
    gradient_checkpointing=False,
    report_to="tensorboard",
    optim="adamw_torch",
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=2, 
            early_stopping_threshold=0.01  
        )
    ]
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.293100,0.896433,0.777000
2,0.161100,0.144145,0.966000
3,0.123800,0.132548,0.981000
4,0.172600,0.074683,0.994000
5,0.038200,0.079148,0.995000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1250, training_loss=0.5243619519233703, metrics={'train_runtime': 2160.9121, 'train_samples_per_second': 18.508, 'train_steps_per_second': 0.578, 'total_flos': 1.052397700529664e+16, 'train_loss': 0.5243619519233703, 'epoch': 5.0})

In [9]:
if "test" in tokenized_dataset: 
    results = trainer.evaluate(tokenized_dataset["test"], metric_key_prefix="test")
    print(f"\nFinal Test Accuracy: {results['test_accuracy']:.4f}")
    print("Full Test Results:", {k: v for k, v in results.items() if k.startswith('test_')})


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


early stopping required metric_for_best_model, but did not find eval_accuracy so early stopping is disabled



Final Test Accuracy: 0.9880
Full Test Results: {'test_loss': 0.17932216823101044, 'test_accuracy': 0.988, 'test_runtime': 15.4337, 'test_samples_per_second': 64.793, 'test_steps_per_second': 1.037}
